In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('train.csv')
trainSup = pd.read_csv('supplemental_train.csv')

In [ ]:
#check what padding does exactly -> adds same value as before, repalce with nan instead
# padding replaces entire data with same values
coin =train.loc[train.Asset_ID == 0].set_index('timestamp')
coin = coin.iloc[0:10]
print(coin)
coin = coin.drop(index = coin.index.to_numpy()[2])
coin = coin.reindex(range(coin.index[0],coin.index[-1]+60,60),method='pad')
print(coin)

In [ ]:
trainSup.timestamp

In [ ]:
def analyseData(df, paddingMethod = 'pad'):
    coins = df.Asset_ID.unique()
    flawedData = np.array([])
    for i in coins:
        coin = df.loc[df.Asset_ID == i].set_index('timestamp')
        coin = coin.reindex(range(coin.index[0],coin.index[-1]+60,60),method=paddingMethod)  #fil missing values
        nan_coin = coin.loc[coin['Target'].isna()]
        flawedData= np.concatenate((flawedData, nan_coin.index.to_numpy()))
        print((nan_coin.index[1:]-nan_coin.index[:-1]).value_counts().head())
        numberTimestamps = coin.index.unique().shape[0]
        overlapTimestamps = coin.shape[0] - numberTimestamps
        print('coin = ',i, coin.shape, coin.index[0], coin.index[len(coin.index)-1], '# of NAN = ', nan_coin.shape[0], nan_coin.shape[0]/coin.shape[0]*100,"%, overlapping index #= ", overlapTimestamps)
        fig = plt.figure()
        plt.scatter(nan_coin.index, np.ones(len(nan_coin.index)))
        plt.title(i)
    print('flawedData in %', np.unique(flawedData).shape[0]/(df.shape[0]/13))
    return flawedData

In [ ]:
flawedDataSup = analyseData(trainSup)

In [ ]:
flawedData = analyseData(train)

In [ ]:
indexListFlawed = np.unique(flawedData)
#print(indexListFlawed)
trainMod = train[train.timestamp.isin(indexListFlawed) == False]
#trainMod = train.drop(index = indexListFlawed, axis = 1)

In [ ]:
trainMod

In [ ]:
def processData(df):
    #iterate over coins
    isInit = False
    coins = train.Asset_ID.unique()
    for coin in coins:
        coinData = df.loc[df.Asset_ID == coin].set_index('timestamp') 
        #print(coinData)
        #pad missing values
        coinData = coinData.reindex(range(coinData.index[0],coinData.index[-1]+60,60),method='pad')
        #print((coinData.index[1:]-coinData.index[:-1]).value_counts().head())
        if not isInit:
            data = pd.DataFrame(coinData)
            isInit = True
        else:
            print(data.shape, coinData.shape)
            data = pd.concat([data,coinData], sort = True)
            
            #print(data.loc[data.index == 1514765100])
    return data
cache = processData(trainMod)

In [ ]:
cache.to_hdf('01_padded_data_padded_nan.h5', key = 'df', mode = 'w')

In [ ]:
def splitDataIntoBatches(df, indexes):
    batches = []
    allIndexes = df.index.to_numpy()
    print(allIndexes)
    print(indexes)
    for i in range(len(indexes)-1):
        currIndex = int(allIndexes[np.where(allIndexes == indexes[i])[0][0]+1])
        nextIndex = int(indexes[i+1])
        print(currIndex, nextIndex)
        print(df.iloc[currIndex], df.iloc[nextIndex])
        batch = df.iloc[currIndex:nextIndex]
        if not batch.empty:
            batches.append(batch)
        print(batch)
        break
    return batches

In [ ]:
flawedDataNotPadded = analyseData(train, None)
print(flawedDataNotPadded)
#coinTest = splitDataIntoBatches(coinData, indexListFlawed)

#print(coinTest[0], indexListFlawed)

In [ ]:
trainModNotPadded = train[train.timestamp.isin(flawedDataNotPadded) == False]

In [ ]:
def addBatchNr(df):
    df['batch'] = 0
    df['batchSize'] = 0
    startBatchIndex = 0
    batchNr = 0
    #print(range(df.shape[0]-1))
    for i in range(df.shape[0]-1):
        diff = df.iloc[i+1].timestamp - df.iloc[i].timestamp
        if diff != 60 or (i+1 == df.shape[0]-1):
            endBatchIndex = i  #iloc 
            df.iloc[startBatchIndex:endBatchIndex+1].batch = batchNr #iloc cuts last index
            df.iloc[startBatchIndex:endBatchIndex+1].batchSize = endBatchIndex-startBatchIndex+1
            startBatchIndex = i+1
            batchNr = batchNr +1
    return df

In [ ]:
def processDataIntoBatches(df):
    #iterate over coins
    isInit = False
    coins = train.Asset_ID.unique()
    for coin in coins:
        coinData = df.loc[df.Asset_ID == coin]#.set_index('timestamp') 
        coinData = addBatchNr(coinData)
        #print(coinData)
        #pad missing values
        if not isInit:
            data = pd.DataFrame(coinData)
            isInit = True
        else:
            print(data.shape, coinData.shape)
            data = pd.concat([data,coinData], sort = True)
            
            #print(data.loc[data.index == 1514765100])
    return data

In [ ]:
trainBatches = processDataIntoBatches(trainModNotPadded)

In [ ]:
trainBatches.to_hdf('03_non_padded_with_batch_info.h5', key = 'df', mode = 'w')

In [ ]:
testData = trainModNotPadded.loc[trainModNotPadded.Asset_ID == 0]#.set_index('timestamp') 
#testData = testData.iloc[0:20]
#print(testData)
#testData = testData.drop([testData.index[2], testData.index[5]])
testDataPadded = addBatchNr(testData)
print(testDataPadded)

In [ ]:
testDataPadded.at[24236570, 'batch'] = 41534
testDataPadded.at[24236570, 'batchSize'] = 1
print(testDataPadded)

In [ ]:
testData = trainModNotPadded.loc[trainModNotPadded.Asset_ID == 10]
print(testData)

In [ ]:
testDataPadded.to_hdf('02_batchSize_to_Timestamp_mapping.h5', key = 'df', mode = 'w')

In [ ]:
np.unique(testDataPadded.batchSize)


In [ ]:
]coinData[1514764920:1514765160]

In [ ]:
test = pd.DataFrame({'team': ['Mavs', 'Lakers', 'Spurs', 'Cavs'],
                   'name': ['Dirk', 'Kobe', 'Tim', 'Lebron'],
                   'rebounds': [11, 7, 14, 7],
                   'points': [26, 31, 22, 29]})
print(test)

batches = splitDataIntoBatches(test, [1,3])
print(batches)